# Milestone 3

## Crime Statistics

Brandon Sams

03May2020

Scraped from https://www.ucrdatatool.gov/Search/Crime/State/RunCrimeStatebyState.cfm

In [175]:
import bs4 as bs
import pandas as pd
import codecs
import numpy as np

In [176]:
pd.set_option('display.max_columns', None)

In [177]:
f_hand = 'Uniform Crime Reporting Statistics.html'
f=codecs.open(f_hand, 'r')
soup = bs.BeautifulSoup(f)

In [178]:
table = soup.find_all("table")[4]

In [179]:
df = pd.DataFrame(columns=range(0,22),index = range(60))

In [180]:
row_marker = 0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        df.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
    row_marker +=1

In [181]:
# Rename Columns
cols = [
    'Year',
    'Population',
    'Violent Crime Total',
    'Murder and Nonnegligent Manslaughter',
    'Legacy Rape',
    'Revised Rape',
    'Robbery',
    'Aggravated Assault',
    'Property Crime Total',
    'Burglary',
    'Larceny-Theft',
    'Motor Vehicle Theft',
    'Violent Crime Rate',
    'Murder and Nonnegligent Manslaughter Rate',
    'Legacy Rape Rate',
    'Revised Rape Rate',
    'Robbery Rate',
    'Aggravated Assault Rate',
    'Property Crime Rate',
    'Burglary Rate',
    'Larceny-Theft Rate',
    'Motor Vehicle Theft Rate'
]

In [182]:
df.columns = cols

In [183]:
# Remove junk rows
df.drop([0,1,2,3,59],inplace = True)

In [184]:
# Index by year
df.set_index('Year',inplace = True)

In [185]:
# Fill Revised Rape columns with NaN
df = df.replace('',np.NaN)

In [186]:
# Convert all columns to numeric type
for col in df:
    df[col] = df[col].str.replace(',','').replace(' ','')

In [187]:
df = df.apply(pd.to_numeric)

In [188]:
df.dtypes

Population                                     int64
Violent Crime Total                            int64
Murder and Nonnegligent Manslaughter           int64
Legacy Rape                                    int64
Revised Rape                                 float64
Robbery                                        int64
Aggravated Assault                             int64
Property Crime Total                           int64
Burglary                                       int64
Larceny-Theft                                  int64
Motor Vehicle Theft                            int64
Violent Crime Rate                           float64
Murder and Nonnegligent Manslaughter Rate    float64
Legacy Rape Rate                             float64
Revised Rape Rate                            float64
Robbery Rate                                 float64
Aggravated Assault Rate                      float64
Property Crime Rate                          float64
Burglary Rate                                f

In [189]:
df

,Population,Violent Crime Total,Murder and Nonnegligent Manslaughter,Legacy Rape,Revised Rape,Robbery,Aggravated Assault,Property Crime Total,Burglary,Larceny-Theft,Motor Vehicle Theft,Violent Crime Rate,Murder and Nonnegligent Manslaughter Rate,Legacy Rape Rate,Revised Rape Rate,Robbery Rate,Aggravated Assault Rate,Property Crime Rate,Burglary Rate,Larceny-Theft Rate,Motor Vehicle Theft Rate
Year,,,,,,,,,,,,,,,,,,,,,
1960,179323175,288460,9110,17190,NaN,107840,154320,3095700,912100,1855400,328200,160.9,5.1,9.6,NaN,60.1,86.1,1726.3,508.6,1034.7,183.0
1961,182992000,289390,8740,17220,NaN,106670,156760,3198600,949600,1913000,336000,158.1,4.8,9.4,NaN,58.3,85.7,1747.9,518.9,1045.4,183.6
1962,185771000,301510,8530,17550,NaN,110860,164570,3450700,994300,2089600,366800,162.3,4.6,9.4,NaN,59.7,88.6,1857.5,535.2,1124.8,197.4
1963,188483000,316970,8640,17650,NaN,116470,174210,3792500,1086400,2297800,408300,168.2,4.6,9.4,NaN,61.8,92.4,2012.1,576.4,1219.1,216.6
1964,191141000,364220,9360,21420,NaN,130390,203050,4200400,1213200,2514400,472800,190.6,4.9,11.2,NaN,68.2,106.2,2197.5,634.7,1315.5,247.4
1965,193526000,387390,9960,23410,NaN,138690,215330,4352000,1282500,2572600,496900,200.2,5.1,12.1,NaN,71.7,111.3,2248.8,662.7,1329.3,256.8
1966,195576000,430180,11040,25820,NaN,157990,235330,4793300,1410100,2822000,561200,220.0,5.6,13.2,NaN,80.8,120.3,2450.9,721.0,1442.9,286.9
1967,197457000,499930,12240,27620,NaN,202910,257160,5403500,1632100,3111600,659800,253.2,6.2,14.0,NaN,102.8,130.2,2736.5,826.6,1575.8,334.1
1968,199399000,595010,13800,31670,NaN,262840,286700,6125200,1858900,3482700,783600,298.4,6.9,15.9,NaN,131.8,143.8,3071.8,932.3,1746.6,393.0


In [190]:
df.index = df.index.astype(int)

In [191]:
from sqlalchemy import create_engine

import pymysql

In [192]:
tableName = 'milestone3'
sqlEngine = create_engine('mysql+pymysql://root:@127.0.0.1/crime', pool_recycle=3600)
dbConnection = sqlEngine.connect()

In [193]:
try:
    frame = df.to_sql(tableName, dbConnection, if_exists='replace');
except ValueError as vx:
    print(vx)
except Exception as ex:   
    print(ex)
else:
    print("Table %s created successfully."%tableName);   
finally:
    dbConnection.close()

Table milestone3 created successfully.


In [194]:
df.dtypes

Population                                     int64
Violent Crime Total                            int64
Murder and Nonnegligent Manslaughter           int64
Legacy Rape                                    int64
Revised Rape                                 float64
Robbery                                        int64
Aggravated Assault                             int64
Property Crime Total                           int64
Burglary                                       int64
Larceny-Theft                                  int64
Motor Vehicle Theft                            int64
Violent Crime Rate                           float64
Murder and Nonnegligent Manslaughter Rate    float64
Legacy Rape Rate                             float64
Revised Rape Rate                            float64
Robbery Rate                                 float64
Aggravated Assault Rate                      float64
Property Crime Rate                          float64
Burglary Rate                                f